In [460]:
import os
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from skimage import transform 


In [461]:
main_folder = r'.\Skany'

images_dict = {}
labels_dict = {}

images_list_arrays = []
label_list_arrays = []

for subfolder in os.listdir(main_folder):   
    subfolder_path = os.path.join(main_folder, subfolder)

    for folder in os.listdir(subfolder_path):
        folder_path = os.path.join(subfolder_path, folder)

        if (folder_path[-4:]) == '.txt':
            f = open(folder_path, "r")
            label = int(f.read())
        elif (folder_path[-4:]) == '.jpg':
            continue
        else:
            images_list = []
            for image_path in os.listdir(folder_path):
                with Image.open(os.path.join(folder_path, image_path)) as img:
                    #img = img.convert('1')
                    img = transform.resize(np.array(img), (224, 224), anti_aliasing=False)
                    images_list.append(img)


            print(np.stack(images_list, axis = 2).shape)                    
            images_dict[folder_path.split('\\')[-1]] = np.concatenate(images_list)
            images_list_arrays.append(np.concatenate(images_list))


            labels_dict[folder_path.split('\\')[-1]] = label
            label_list_arrays.append(label)


(224, 224, 8, 3)
(224, 224, 8, 3)
(224, 224, 8, 3)
(224, 224, 8, 3)
(224, 224, 8, 3)
(224, 224, 8, 3)
(224, 224, 8, 3)


In [462]:
main_folder = r'.\Skany'

images_dict = {}
labels_dict = {}

images_list_arrays = []
label_list_arrays = []

for subfolder in os.listdir(main_folder):   
    subfolder_path = os.path.join(main_folder, subfolder)

    for folder in os.listdir(subfolder_path):
        folder_path = os.path.join(subfolder_path, folder)

        if (folder_path[-4:]) == '.txt':
            f = open(folder_path, "r")
            label = int(f.read())
        elif (folder_path[-4:]) == '.jpg':
            continue
        else:
            images_list = []
            for image_path in os.listdir(folder_path):
                with Image.open(os.path.join(folder_path, image_path)) as img:
                    #img = img.convert('1')
                    img = transform.resize(np.array(img), (224, 224), anti_aliasing=False)
                    images_list.append(img)
                break

            images_dict[folder_path.split('\\')[-1]] = (img)
            images_list_arrays.append((img))


            labels_dict[folder_path.split('\\')[-1]] = label
            label_list_arrays.append(label)


In [463]:
# images_dict[folder_path.split('\\')[-1]] = np.concatenate(images_list)
#             images_list_arrays.append(np.concatenate(images_list))


In [464]:
numpy_images = np.stack(images_list_arrays, axis = 0)
numpy_labels = np.stack(label_list_arrays, axis = 0)


In [465]:
numpy_images.shape, numpy_labels.shape

((7, 224, 224, 3), (7,))

In [466]:
numpy_labels

array([1, 0, 1, 0, 1, 0, 1])

In [467]:
x = preprocess_input(numpy_images)


In [468]:
x.shape

(7, 224, 224, 3)

In [469]:
# numpy_images = np.transpose(numpy_images, (0, 3, 1, 2))

In [470]:
numpy_images.shape

(7, 224, 224, 3)

In [471]:
from tensorflow.keras.layers import Flatten, Dense, GlobalAveragePooling2D


In [472]:
model = keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    classes=2,
    pooling = 'avg',
    classifier_activation="softmax",
)


In [473]:
from tensorflow.keras.utils import to_categorical

In [474]:
numpy_labels = to_categorical(numpy_labels, 2)

In [475]:
numpy_labels.shape

(7, 2)

In [476]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_96 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_96[0][0]']            
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [477]:
from tensorflow.keras.metrics import categorical_crossentropy

In [478]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.Accuracy()])

# Train end-to-end. Be careful to stop before you overfit!
model.fit(x=x, y=numpy_labels, epochs=1)


ValueError: in user code:

    File "c:\Users\krukd\anaconda3\envs\magisterka_grzechu\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\krukd\anaconda3\envs\magisterka_grzechu\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\krukd\anaconda3\envs\magisterka_grzechu\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\krukd\anaconda3\envs\magisterka_grzechu\Lib\site-packages\keras\src\engine\training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "c:\Users\krukd\anaconda3\envs\magisterka_grzechu\Lib\site-packages\keras\src\engine\training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_wei